In [1]:
%gui qt

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace
)
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Dir, gp_Pln, gp_Vec
from OCC.Display.SimpleGui import init_display

import numpy as np
import json


DSTV_FACE_MAP = {'I':['O','U','V'], 'U':['H','U','O'], 'L':['H','U']}

# Start display without blocking
display, start_display, add_menu, add_function_to_menu = init_display()

# JSON Library path
json_path = "../data/Shape_classifier_info.json"

display.EraseAll()
display.View_Iso()
display.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
def compute_obb(shape):
    """
    Compute the Oriented Bounding Box for a shape.
    Returns a Bnd_OBB instance.
    """
    obb = Bnd_OBB()
    brepbndlib.AddOBB(shape, obb)
    return obb

In [4]:
def transform_to_global_origin(shape, obb):
    """
    Rotate solid to align with global axes and move start face to (0,0,0).
    """

    # Step 1: Rotation alignment using OBB axes
    c     = obb.Center()
    x_vec = obb.XDirection()
    y_vec = obb.YDirection()
    z_vec = obb.ZDirection()
    x_d   = gp_Dir(x_vec)
    y_d   = gp_Dir(y_vec)
    z_d   = gp_Dir(z_vec)

    localCS  = gp_Ax3(gp_Pnt(c.X(), c.Y(), c.Z()), z_d, x_d)
    globalCS = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

    trsf_rot = gp_Trsf()
    trsf_rot.SetTransformation(localCS, globalCS)
    rotated = BRepBuilderAPI_Transform(shape, trsf_rot, True).Shape()

    # Step 2: Translate so the min-Z corner lies at origin
    obb_rotated = Bnd_OBB()
    brepbndlib.AddOBB(rotated, obb_rotated)

    center_rot = obb_rotated.Center()
    z_axis = obb_rotated.ZDirection()
    x_axis = obb_rotated.XDirection()
    y_axis = obb_rotated.YDirection()
    hx, hy, hz = obb_rotated.XHSize(), obb_rotated.YHSize(), obb_rotated.ZHSize()

    min_corner = gp_Pnt(
        center_rot.X() - hx * x_axis.X() - hy * y_axis.X() - hz * z_axis.X(),
        center_rot.Y() - hx * x_axis.Y() - hy * y_axis.Y() - hz * z_axis.Y(),
        center_rot.Z() - hx * x_axis.Z() - hy * y_axis.Z() - hz * z_axis.Z()
    )

    shift_vec = gp_Vec(min_corner, gp_Pnt(0, 0, 0))
    trsf_shift = gp_Trsf()
    trsf_shift.SetTranslation(shift_vec)

    transformed = BRepBuilderAPI_Transform(rotated, trsf_shift, True).Shape()
    return transformed


In [5]:
def rotate_z_90(shape):
    """
    Rotates the shape 90 degrees about the global Z axis.
    """
    from OCC.Core.gp import gp_Trsf
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
    from math import pi

    trsf = gp_Trsf()
    trsf.SetRotation(gp_Ax3().Axis(), pi / 2.0)  # 90° about Z
    rotated = BRepBuilderAPI_Transform(shape, trsf, True).Shape()
    return rotated

In [6]:
def fingerprint_shape(shape, obb):
    """
    Compute width, height, and CSA of the cross-section.
    Attempts wire-based CSA first; falls back to building from edges if needed.
    """
    from OCC.Core.TopExp import TopExp_Explorer
    from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire

    half_sizes = [obb.XHSize(), obb.YHSize(), obb.ZHSize()]
    length_idx = max(range(3), key=lambda i: half_sizes[i])
    dims = [2 * h for h in half_sizes]
    width, height = sorted([dims[i] for i in range(3) if i != length_idx], reverse=True)

    center_xyz = obb.Center()
    center = gp_Pnt(center_xyz.X(), center_xyz.Y(), center_xyz.Z())
    dir_map = [obb.XDirection(), obb.YDirection(), obb.ZDirection()]
    normal = gp_Dir(dir_map[length_idx])
    plane = gp_Pln(center, normal)

    section = BRepAlgoAPI_Section(shape, plane, True)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    compound = section.Shape()

    # Try wire-based approach first
    exp = TopExp_Explorer(compound, TopAbs_WIRE)
    if exp.More():
        wire = exp.Current()
    else:
        # Fallback: build wire from edges
        edge_exp = TopExp_Explorer(compound, TopAbs_EDGE)
        wire_builder = BRepBuilderAPI_MakeWire()
        edge_count = 0
        while edge_exp.More():
            edge = edge_exp.Current()
            wire_builder.Add(edge)
            edge_exp.Next()
            edge_count += 1

        if edge_count == 0:
            raise RuntimeError("No wire or edge found in section result")
        if not wire_builder.IsDone():
            raise RuntimeError("Failed to build wire from edges")

        wire = wire_builder.Wire()

    # Build face from wire and compute area
    face = BRepBuilderAPI_MakeFace(plane, wire, True).Face()
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    area = props.Mass()
    return width, height, area


In [7]:
def visualize(shape, obb, display):
    """
    Display shape aligned to global origin, with axes, on the provided viewer.
    """
    transformed = transform_to_global_origin(shape, obb)
    # Determine scale for axes from OBB extents
    dims = [2*obb.XHSize(), 2*obb.YHSize(), 2*obb.ZHSize()]
    scale = max(dims) * 0.5
    # Show aligned solid
    display.DisplayShape(transformed, update=False)
    # Draw axes
    origin = gp_Pnt(0, 0, 0)
    for vec, color in [((scale,0,0), "RED"), ((0,scale,0), "GREEN"), ((0,0,scale), "BLUE")]:
        edge = BRepBuilderAPI_MakeEdge(origin, gp_Pnt(*vec)).Edge()
        display.DisplayShape(edge, color=color, update=False)



In [8]:
def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to handle OBB axis flips.
    """
    with open(json_path) as f:
        lib = json.load(f)
    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        **info,
                        "Designation": name,
                        "Category": cat,
                        "Measured_height": height,
                        "Measured_width": width,
                        "Measured_area": cs["area"],
                        "Measured_length": cs["length"],
                        "Match_score": score,
                        "Profile_type": cs.get("profile_type", "Unknown")
                    }
        return best, best_score

    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    if original and (not swapped or score1 <= score2):
        swapped["Swapped_dimensions"] = False
        return original
    elif swapped:
        swapped["Swapped_dimensions"] = True
        return swapped
    else:
        return None

In [9]:
def build_classification_input(obb, width, height, area):
    """
    Constructs a classification-friendly dictionary using globally aligned OBB.
    Assumes the solid is aligned with Z as the length direction.
    """
    length = 2 * obb.ZHSize()
    return {
        "span_web": height,
        "span_flange": width,
        "area": area,
        "length": length,
    }

In [10]:
if __name__ == "__main__":

    # step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step"
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    step_path = "../data/TestPFC.step"
    
    # Load shape and compute OBB once
    shape_orig = load_step(step_path)
    obb_orig = compute_obb(shape_orig)
    shape_aligned = transform_to_global_origin(shape_orig, obb_orig)
    obb_aligned = compute_obb(shape_aligned)

    
    visualize(shape_aligned, obb_aligned, display)
    display.FitAll()

    width, height, area = fingerprint_shape(shape_aligned, obb_aligned)
    print(f"Width: {width:.3f}, Height: {height:.3f}, CSA: {area:.3f}")

    print("OBB extents before rotation:")
    print("X:", 2 * obb_aligned.XHSize())
    print("Y:", 2 * obb_aligned.YHSize())
    print("Z:", 2 * obb_aligned.ZHSize())

    cs = build_classification_input(obb_aligned, width, height, area)

    result = classify_profile(cs, json_path)

    print(result)


Many colors for color name BLUE, using first.
Width: 100.122, Height: 50.054, CSA: 1299.766
OBB extents before rotation:
X: 50.053692161039095
Y: 100.1224026380103
Z: 1000.0135754058629
{'mass': 10.20315915547705, 'height': 100.0, 'width': 50.0, 'csa': 1300.0, 'web_thickness': 5.0, 'flange_thickness': 8.5, 'root_radius': 9.0, 'toe_radius': 0.0, 'code_profile': 'U', 'Designation': '100x50x10', 'Category': 'PFC', 'Measured_height': 100.1224026380103, 'Measured_width': 50.053692161039095, 'Measured_area': 1299.7655045080442, 'Measured_length': 1000.0135754058629, 'Match_score': 0.1941329138152285, 'Profile_type': 'Unknown', 'Swapped_dimensions': True}


In [11]:
if result.get("Swapped_dimensions"):
    print("Swapped dimensions detected — rotating 90° about Z.")
    shape_aligned = rotate_z_90(shape_aligned)
    obb_aligned = compute_obb(shape_aligned) 

    #clear display and show new orientation
    display.EraseAll()  # clear previous shapes and axes
    visualize(shape_aligned, obb_aligned, display)
    display.FitAll()

Swapped dimensions detected — rotating 90° about Z.
Many colors for color name BLUE, using first.


In [12]:
# Rerun fingerprint & classifier to confirm results

width, height, area = fingerprint_shape(shape_aligned, obb_aligned)
print(f"Width: {width:.3f}, Height: {height:.3f}, CSA: {area:.3f}")
cs = build_classification_input(obb_aligned, width, height, area)
result = classify_profile(cs, json_path)

print("OBB extents after rotation:")
print("X:", 2 * obb_aligned.XHSize())
print("Y:", 2 * obb_aligned.YHSize())
print("Z:", 2 * obb_aligned.ZHSize())

print(result)

Width: 100.122, Height: 50.054, CSA: 1299.766
OBB extents after rotation:
X: 50.05369216103912
Y: 100.12240263801037
Z: 1000.0135754058629
{'mass': 10.20315915547705, 'height': 100.0, 'width': 50.0, 'csa': 1300.0, 'web_thickness': 5.0, 'flange_thickness': 8.5, 'root_radius': 9.0, 'toe_radius': 0.0, 'code_profile': 'U', 'Designation': '100x50x10', 'Category': 'PFC', 'Measured_height': 100.12240263801037, 'Measured_width': 50.05369216103912, 'Measured_area': 1299.7655003157045, 'Measured_length': 1000.0135754058629, 'Match_score': 0.19413323630299736, 'Profile_type': 'Unknown', 'Swapped_dimensions': True}
